In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

In [3]:
file=r'C:\Users\ARUNA\Deep Learning Edvancer\Download Data\sentiment labelled sentences/imdb_labelled.txt'

In [4]:
imdb=pd.read_csv(file,sep='\t',header=None,names=['review','sentiment'])

In [5]:
# imdb

In [6]:
imdb_train,imdb_test=train_test_split(imdb,test_size=0.2,random_state=2)

In [7]:
x_train=imdb_train['review']
y_train=imdb_train['sentiment']
x_test=imdb_test['review']
y_test=imdb_test['sentiment']

In [9]:
sent_lens=[]
for sent in imdb_train['review']:
    sent_lens.append(len(word_tokenize(sent)))
    
    
    

In [10]:
max(sent_lens)

1616

In [11]:
# sent_lens

In [12]:
np.quantile(sent_lens,0.95)


40.0

In [13]:
# We can see that 95% review text are of lengths less than or equal to 37. We'll keep the max length to 30

In [15]:
max_len = 40

tok = Tokenizer(char_level=False,split=' ')

tok.fit_on_texts(x_train)


In [16]:
tok.index_word

{1: 'the',
 2: 'a',
 3: 'and',
 4: 'of',
 5: 'is',
 6: 'this',
 7: 'it',
 8: 'i',
 9: 'to',
 10: 'was',
 11: 'movie',
 12: 'in',
 13: 'film',
 14: 'that',
 15: '1',
 16: '0',
 17: 'but',
 18: 'for',
 19: 'as',
 20: 'with',
 21: 'are',
 22: 'on',
 23: 'not',
 24: 'you',
 25: 'one',
 26: 'very',
 27: 'bad',
 28: 'just',
 29: 'so',
 30: 'good',
 31: 'all',
 32: 'an',
 33: "it's",
 34: 'there',
 35: 'be',
 36: 'by',
 37: 'about',
 38: 'at',
 39: 'if',
 40: 'out',
 41: 'great',
 42: 'his',
 43: 'from',
 44: 'like',
 45: 'have',
 46: 'time',
 47: 'were',
 48: 'well',
 49: 'has',
 50: 'even',
 51: 'really',
 52: 'my',
 53: 'or',
 54: 'who',
 55: 'acting',
 56: 'he',
 57: 'when',
 58: 'most',
 59: 'see',
 60: 'how',
 61: 'more',
 62: 'characters',
 63: 'would',
 64: 'no',
 65: 'only',
 66: 'ever',
 67: 'made',
 68: 'also',
 69: 'best',
 70: '10',
 71: 'plot',
 72: 'some',
 73: 'your',
 74: 'do',
 75: 'its',
 76: 'character',
 77: 'real',
 78: 'because',
 79: 'love',
 80: "didn't",
 81: 'movies

In [17]:
sequences_train = tok.texts_to_sequences(x_train)

In [19]:
sequences_train


[[34, 146, 21, 30, 90, 426],
 [274, 592, 3, 1, 335, 10, 275, 94],
 [52, 593, 70, 427, 5, 237, 18, 1, 71],
 [6,
  984,
  5,
  276,
  985,
  428,
  8,
  594,
  2,
  429,
  4,
  210,
  595,
  9,
  35,
  1,
  336,
  430,
  13,
  66,
  67],
 [8,
  183,
  59,
  60,
  6,
  11,
  82,
  35,
  32,
  986,
  9,
  131,
  9,
  277,
  40,
  53,
  987,
  596,
  3,
  988],
 [8,
  431,
  132,
  114,
  61,
  8,
  83,
  44,
  432,
  29,
  8,
  83,
  278,
  9,
  35,
  25,
  17,
  8,
  238,
  6,
  13,
  5,
  115,
  73,
  46],
 [39, 24, 59, 7, 24, 184, 279, 28, 597, 7, 22, 1, 989],
 [1, 239, 337, 162, 990, 1, 991, 4, 1, 13],
 [1,
  106,
  63,
  598,
  20,
  599,
  600,
  433,
  992,
  9,
  280,
  84,
  14,
  63,
  35,
  126,
  163,
  1,
  993,
  3,
  994,
  18],
 [995,
  996,
  5,
  2,
  26,
  997,
  998,
  601,
  54,
  82,
  35,
  26,
  999,
  38,
  338,
  3,
  339,
  164,
  28,
  434,
  44,
  2,
  1000,
  340,
  17,
  1001,
  12,
  95,
  1002,
  18,
  602,
  3,
  1003],
 [68,
  603,
  5,
  281,
  1004,
  2

In [20]:
vocab_len=len(tok.index_word.keys())

In [21]:
vocab_len

2688

In [22]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [24]:
sequences_matrix_train

array([[  0,   0,   0, ...,  30,  90, 426],
       [  0,   0,   0, ...,  10, 275,  94],
       [  0,   0,   0, ...,  18,   1,  71],
       ...,
       [  0,   0,   0, ...,  44,  64,  84],
       [  0,   0,   0, ..., 284, 284, 284],
       [  0,   0,   0, ..., 383,  20, 307]])

In [25]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    
    layer = Embedding(vocab_len+1,500,input_length=max_len,
                      mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model



In [26]:
model = RNN()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 40)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 40, 500)           1344500   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                144640    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [27]:
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [28]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [34]:
model.fit(sequences_matrix_train,y_train.values,batch_size=50,
          epochs=6,validation_data=(sequences_matrix_test,y_test.values))

Train on 598 samples, validate on 150 samples
Epoch 1/6
598/598 [==============================] - 3s 4ms/sample - loss: 3.7896e-05 - accuracy: 1.0000 - val_loss: 2.2335 - val_accuracy: 0.7533
Epoch 2/6
598/598 [==============================] - 2s 3ms/sample - loss: 3.3414e-05 - accuracy: 1.0000 - val_loss: 2.2457 - val_accuracy: 0.7533
Epoch 3/6
598/598 [==============================] - 1s 3ms/sample - loss: 3.0939e-05 - accuracy: 1.0000 - val_loss: 2.2559 - val_accuracy: 0.7533
Epoch 4/6
598/598 [==============================] - 2s 3ms/sample - loss: 2.4690e-05 - accuracy: 1.0000 - val_loss: 2.2650 - val_accuracy: 0.7533
Epoch 5/6
598/598 [==============================] - 2s 3ms/sample - loss: 3.1023e-05 - accuracy: 1.0000 - val_loss: 2.2742 - val_accuracy: 0.7533
Epoch 6/6
598/598 [==============================] - 1s 2ms/sample - loss: 2.8723e-05 - accuracy: 1.0000 - val_loss: 2.2830 - val_accuracy: 0.7533


In [35]:
predictions=model.predict(sequences_matrix_test)

In [36]:
from sklearn.metrics import roc_auc_score

In [37]:
roc_auc_score(y_test,predictions)

0.8383431740919663

In [38]:
# if you notice the validation loss , it is actually increasing after some epochs , best model appears much earlier than 
# 50th epoch